# REGRESSION PAR DISCONTINUITE

Cette application utilise les données de l'article de Basten et Betz (2013) : "Beyond Work Ethic: Religion, Individual, and Political Preferences", *American Economic Journal: Economic Policy*, 5(3):67-91.

En 1904, Max Weber, dans son célèbre ouvrage L'éthique protestante et l'esprit du capitalisme (1904), situe l'essor du capitalisme dans la religion protestante. Selon son texte, à l'origine, la religion protestante voit dans l'enrichissement personnel le signe de l'élection divine. Cette recherche du signe électif par les protestants serait à l'origine d'une éthique bien particulière du travail, fondée sur l'effort, l'accumulation et le mérite personnel.

On veut tester si la religion protestante a vraiment un impact sur l'éthique du travail (= l'importance donnée à l'effort au travail). 

Pour identifier l'effet causal du protestantisme sur l’éthique du travail, on s'intéresse au cas de la Suisse , au sein de laquelle coexistent des religions majoritaires différentes dans les différents cantons : protestantisme, ou catholicisme.

# 1. Introduction

De manière générale, pourquoi est-il très difficile de mesurer l'impact causal d'une religion sur un résultat économique ? 


# 2 Données

On ouvre les donnée : **TD11_weber.dta** :

In [ ]:
library(foreign)
d<-read.dta("TD11_weber.dta")
head(d)

Voici le *codebook* :
![](TD11_codebook.png)

Quel est le niveau d'observation de la base de données (= à quoi correspond chaque ligne de la base de données) : individu ? municipalité ? canton ? pays ?

Quelle est la mesure de la religion ? Comment peut-on mesurer l'éthique du travail ? 

Produire un graphique mettant en relation cette mesure de la religion (absisse) avec la mesure de préférence pour le loisir (ordonnée).

In [ ]:
library(ggplot2)
ggplot(d, aes(x=prot_share,y=pfl))+geom_point()

Que constate-t-on ? Est-ce compatible avec la théorie de Weber selon laquelle les protestants ont une plus forte éthique du travail ?

# 3. Stratégie d'identification

Pour nous aider à définir une stratégie, voilà ce que nous savons sur la religion des différents cantons en Suisse : 

- Les cantons les plus ruraux et montagneux sont catholiques, les cantons les plus urbains sont protestants.
- Les cantons germanophones sont protestants, les cantons francophones et italophones sont catholiques. 
- Les cantons qui étaient corporatistes (plus de mobilité sociale) ont plutôt choisi le protestantisme, les cantons avec aristocratie sont restés catholiques. 

Les auteurs pensent d'abord à comparer les préférences des municipalités suisses en fonction de leur pourcentage de protestants. Pourquoi est-ce une mauvaise idée ? 

Dans le sud-est de la Suisse d'aujourd'hui, il existe une région francophone homogène avant 1530, qui perd un conflit et se trouve partagée en deux : 

- une partie intègre un canton catholique : canton de Fribourg (127 municipalités), 
- une autre intègre de force un canton protestant : canton de Vaud (382 municipalités). 

a) Quelle stratégie d'identification proposeriez-vous ? 

b) Quel est alors le traitement ? La variable dépendante ? 

c) Quand a eu lieu l'assignation au traitement ? A priori, l'adhésion est-elle parfaite ? 

d) En conséquence, quel modèle estimeriez vous, avec quel estimateur ?

e) Il s'avère que nos données **TD11_weber** se concentrent justement déjà sur l'ensemble des municipalités du canton de Vaud et de Fribourg. Estimer le modèle que vous proposez. 

In [ ]:
##si l'éadhésion avait été parfaite : 
#simple2<-lm(pfl~vaud,data=d)
#summary(simple2)

#comme l'adhésion est imparfaite : 
library(AER)
iv<-ivreg(pfl~prot_share|vaud,data=d)
summary(iv)

# 3. Estimation : régression par discontinuité

Les chercheurs remarquent que malgré leur homogénéité de prime abord, les cantons de Fribourg et Vaud ont quand même des différences géographiques : l'un est plus montagneuse que l'autre. Or historiquement les régions montagneuses sont plus égalitaires en Suisse. Ceci pourrait biaiser les résultats. 

Les auteurs décident d'affiner encore leur stratégie et de se concentrer sur la frontière entre ces deux régions, et proposent une **régression par discontinuité spatiale**. 

### 3.1 Questions préliminaires

Quel est le principe d'identification ?

Quel est le seuil ? Quelle est la running variable ? Définir une fenêtre de votre choix autour du seuil. 

### 3.2 Sharp design

On suppose d'abord que l'adhésion est parfaite : on imagine que les municipalités d'un côté de la frontière sont toutes à 100% protestantes, et de l'autre à 100% catholiques.

On va donc estimer un modèle de régression par discontinuité *sharp*. On commence par l'estimer séparément d'un côté et de l'autre du seuil. 

In [ ]:
vaud0<-lm(pfl~borderdis,data=d[d$vaud==0 & d$borderdis<10 & d$borderdis>-10,])
vaud1<-lm(pfl~borderdis,data=d[d$vaud==1 & d$borderdis<10 & d$borderdis>-10,])

Puis on utilise les coefficients estimés pour calculer l'effet du protestantisme sur la préférence pour le loisir : 

In [ ]:
effet<-coef(vaud1)[1]-coef(vaud0)[1]
effet

Retrouvez ce même résultat en estimant le modèle en une seule équation.

In [ ]:
rdd_sharp<-lm(pfl~vaud+borderdis+vaud:borderdis,data=d[d$borderdis<10&d$borderdis>-10,])
summary(rdd_sharp)

Conclure.

### 3.3 Fuzzy design

On sait que l'adhésion est imparfaite : en réalité, le design est *fuzzy*. Les communes nes sont pas toutes à 100% protestantes ou catholiques selon le côté de la frontière. Sachant cela, qu'a-t-on en fait estimé en 3.2 ? 

Estimer le modèle de régression fuzzy.

In [ ]:
library(AER)
rdd_fuzzy<-ivreg(pfl~prot_share+borderdis+vaud:borderdis|vaud+borderdis+vaud:borderdis,data=d[d$borderdis<10&d$borderdis>-10,])
summary(rdd_fuzzy)

Conclure. 

# 4. Analyse graphique

Grâce au cours, on sait que les régressions par discontinuité sont particulièrement parlantes lorsqu'on présente les résultats graphiquement. 

1) Que permet de montrer le graphique issu du code ci-dessous ?

In [ ]:
library(ggplot2)
d$Annee1530<-factor(d$vaud)
levels(d$Annee1530)<-c("Catholique","Protestant")

ggplot(d, aes(x = borderdis, y = prot_share,colour=Annee1530)) + 
  geom_point() + ylim(0,1) + stat_smooth(method = "lm")


2) Produire un graphique dans le même esprit pour montrer l'effet de la décision de 1530 sur les scores aux référendas des différentes municipalités (=l'effet d'intention de traiter).

In [ ]:
ggplot(d, aes(x = borderdis, y = pfl,colour=Annee1530)) + 
  geom_point() + ylim(25,60) + stat_smooth(method = "lm")

3) Quel serait l'intérêt de produire le même graphique que (2) mais en remplaçant l'outcome et en utilisant des variables de contrôle à sa place, comme *income* ou encore *altitude* ?

In [ ]:
#OPTIONNEL : 
ggplot(d, aes(x = borderdis, y = income,colour=Annee1530)) + 
  geom_point() + stat_smooth(method = "lm")
ggplot(d, aes(x = borderdis, y = altitude,colour=Annee1530)) + 
  geom_point() + stat_smooth(method = "lm")

4. Changer l'option de stat_smooth() **lm** par **loess**. Commentez.

In [ ]:
ggplot(d, aes(x = borderdis, y = prot_share,colour=Annee1530)) + 
  geom_point() + ylim(0,1) + stat_smooth(method = "loess")

<font color=blue> NOTE: LOESS est une méthode de régression non paramétrique, qui permet de fitter des données de manière non linéaire. 

Extrait de définition sur WIKIPEDIA : La régression locale, ou LOESS, est une méthode de régression non paramétrique fortement connexe qui combine plusieurs modèles de régression multiple au sein d'un méta-modèle qui repose sur la méthode des k plus proches voisins. « LOESS » est un acronyme qui peut être compris comme signifiant, en anglais, « LOcally weighted Scatterplot Smoother ».

La régression locale est une alternative possible aux méthodes habituelles de régression, comme la régression par les moindres carrés linéaire ou non linéaire, dans les cas où ces dernières s'avèrent mal adaptées. Elle combine la simplicité de régression linéaire par les moindres carrés avec la flexibilité de la régression non linéaire, en effectuant une régression simple sur des sous-ensembles locaux de données. L'un des principaux avantages de cette méthode est qu'elle rend inutile la définition d'une unique fonction globale qui décrirait le modèle de régression, puisque la méthode consiste à calculer autant de fonctions locales qu'il y a de segments de données. 

</font>